# **DIVE INTO CODE COURSE**
## **U-Net**
**Student Name**: Doan Anh Tien<br>
**Student ID**: 1852789<br>
**Email**: tien.doan.g0pr0@hcmut.edu.vn

### **[Problem 1] Learning/Estimation**

In [7]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


**Dataset Preparation**

Sort the list of images of both train and test dataset (rename the train and label images)

In [8]:
import glob
from PIL import Image

In [11]:
data_path = '/content/drive/MyDrive/Colab Notebooks/DIVE INTO CODE/Term 3/unet-master/data/membrane/train/image'

files = glob.glob(data_path + "/*")
files.sort()

for i, pic in enumerate(files):
  name = "image_" + str(i) + ".png"
  os.rename(pic, os.path.join(data_path, name))

In [12]:
label_path = '/content/drive/MyDrive/Colab Notebooks/DIVE INTO CODE/Term 3/unet-master/data/membrane/train/label'

files = glob.glob(label_path + "/*")
files.sort()

for i, pic in enumerate(files):
  name = "label_" + str(i) + ".png"
  os.rename(pic, os.path.join(label_path, name))

In [14]:
test_path = '/content/drive/MyDrive/Colab Notebooks/DIVE INTO CODE/Term 3/unet-master/data/membrane/test'

files = glob.glob(test_path + "/*")
files.sort()

for i, pic in enumerate(files):
    name = str(i) + ".png"
    os.rename(pic, os.path.join(test_path, name))

As the input size is (256, 256) as default, we will change the resolution of the provided dataset. For album expansion, the U-Net implementation has done it already for us in the main.py file, so we don't need to do it in this case.

In [15]:
files = glob.glob(data_path + "/*")

for pic in files:
  photo = Image.open(pic)
  photo_resize = photo.resize((256, 256))
  photo_resize.save(pic)


files = glob.glob(label_path + "/*")

for pic in files:
  photo = Image.open(pic)
  photo_resize = photo.resize((256, 256))
  photo_resize.save(pic)

files = glob.glob(test_path + "/*")

for pic in files:
  photo = Image.open(pic)
  photo_resize = photo.resize((256, 256))
  photo_resize.save(pic)

**Train and Estimate**

In [16]:
%cd /content/drive/MyDrive/Colab Notebooks/DIVE INTO CODE/Term 3/unet-master

/content/drive/MyDrive/Colab Notebooks/DIVE INTO CODE/Term 3/unet-master


In [17]:
!ls

data		   img	    model.py	 trainUnet.ipynb
dataPrepare.ipynb  LICENSE  __pycache__  unet_membrane.hdf5
data.py		   main.py  README.md


In [4]:
!pip install keras==2.1.5

     |████████████████████████████████| 334 kB 5.3 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0


In [14]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 24 kB/s 
     |████████████████████████████████| 503 kB 42.0 MB/s 
     |████████████████████████████████| 3.8 MB 29.7 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=4e104fd168460721ecd3e0b6c340f8e8c4dd229d04c4f32ea15e8c54fd129adb
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [18]:
!python main.py

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/drive/MyDrive/Colab Notebooks/DIVE INTO CODE/Term 3/unet-master/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


2022-01-29 14:01:24.738746: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-01-29 14:01:24.751396: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2022-01-29 14:01:24.751685: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ee8b148a00 initialized for platf

### **[Problem 2] Code reading**

From the source code, main.py and model.py can be considered as the two main files to construct the U-Net convolutional architecture and perform the learning progress.

As for the main.py, we have:

**1. trainGenerator**

A function generate image and mask with configured settings at the same time, and the seed is used to ensure the consistent of the dataset.

The function takes in the batch size, the directories for the train and label data, the dictionary contain the parameters to transform the images, or the number of classes to be predict and the other parameters like color mode or target image shape.

The configuration dictionary can be used as an example below (from the source code):
```python
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
```

**2. unet**

The function that form an U-Net convolutional deep neural network, it will be specifically described in the model.py file.

**3. ModelCheckpoint**

The function that monitor the loss and save the model checkpoint as a hdf5 format for each epochs.

**4. fit_generator**

The built-in function of the keras model that will start the learning stage.

**5. testGenerator**

A function that generate the test dataset. Work similarly to trainGenerator function

**6. predict_generator**

The function that let the model predict the test dataset based on what it has learned.

**7. saveResult**

Save estimated results to the path.



For the model.py, here lies the implementation of the U-Net architecture:

```python
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model
```


The model takes the image with the shape of (256, 256) as default and use a combination of Conv and Pooling from the keras library to perform image segmentation. The first half of the model performs the convolution, and the second half left performs the deconvolution.

The activation function used throughout the model is ReLU, and for the final layer, sigmoid is used for the binary classification. The model does not use padding at all.

For the loss function, the U-Net uses binary cross-entropy and the Adam optimizer. The accuracy is chosen to be the main metric.